<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Explanations/blob/main/notebooks/test/CNL_Q600G4F15_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_Q_fourth15.zip -d /content/temp
! mv /content/temp/content/dataset/test /content/test
! mv /content/temp/content/dataset/annotations.txt /content/annotations.txt
! rm -rf /content/temp

CPU times: user 607 ms, sys: 117 ms, total: 723 ms
Wall time: 1min 31s


In [3]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10884, done.
remote: Counting objects: 100% (411/411), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 10884 (delta 243), reused 300 (delta 197), pack-reused 10473
Receiving objects: 100% (10884/10884), 20.51 MiB | 19.72 MiB/s, done.
Resolving deltas: 100% (7958/7958), done.


In [13]:
! python -u ./pytorch-image-models/inference.py test -b 32 --input-size 3 600 600 --num-classes 57 --no-test-pool --checkpoint /content/drive/MyDrive/UNC/S2022P/output/20220822-234656-convnext_large_384_in22ft1k-600/last.pth.tar --model convnext_large_384_in22ft1k

Loaded state_dict from checkpoint '/content/drive/MyDrive/UNC/S2022P/output/20220822-234656-convnext_large_384_in22ft1k-600/last.pth.tar'
Model convnext_large_384_in22ft1k created, param count: 196317945
Predict: [0/539] Time 7.215 (7.215)
Predict: [10/539] Time 1.368 (1.907)
Predict: [20/539] Time 1.386 (1.657)
Predict: [30/539] Time 1.405 (1.571)
Predict: [40/539] Time 1.388 (1.527)
Predict: [50/539] Time 1.381 (1.500)
Predict: [60/539] Time 1.371 (1.481)
Predict: [70/539] Time 1.393 (1.468)
Predict: [80/539] Time 1.404 (1.458)
Predict: [90/539] Time 1.387 (1.451)
Predict: [100/539] Time 1.402 (1.446)
Predict: [110/539] Time 1.378 (1.441)
Predict: [120/539] Time 1.392 (1.437)
Predict: [130/539] Time 1.381 (1.434)
Predict: [140/539] Time 1.380 (1.433)
Predict: [150/539] Time 1.380 (1.431)
Predict: [160/539] Time 1.389 (1.430)
Predict: [170/539] Time 1.407 (1.428)
Predict: [180/539] Time 1.387 (1.426)
Predict: [190/539] Time 1.383 (1.425)
Predict: [200/539] Time 1.394 (1.423)
Predict: 

In [16]:
from glob import glob
import os
from collections import defaultdict

class_dict = {}
for fn in glob("test/**/*.*", recursive=True):
  cat = int(fn.split("/")[-2])
  bn = os.path.basename(fn)

  class_dict[bn] = cat

In [17]:
genera_dict = {}
species_dict = {}
anno_file = open("annotations.txt")
anno_lines = anno_file.readlines()

for l in anno_lines:
  l = l.strip().split(",")
  cat, species, genus = l[0], l[1], l[1].split(" ")[0]

  genera_dict[int(cat)] = genus
  species_dict[int(cat)] = species

In [18]:
print(genera_dict.items())

dict_items([(0, 'Quercus'), (1, 'Quercus'), (2, 'Quercus'), (3, 'Quercus'), (4, 'Quercus'), (5, 'Quercus'), (6, 'Quercus'), (7, 'Quercus'), (8, 'Quercus'), (9, 'Quercus'), (10, 'Quercus'), (11, 'Quercus'), (12, 'Quercus'), (13, 'Quercus'), (14, 'Quercus'), (15, 'Quercus'), (16, 'Quercus'), (17, 'Quercus'), (18, 'Quercus'), (19, 'Quercus'), (20, 'Quercus'), (21, 'Quercus'), (22, 'Quercus'), (23, 'Quercus'), (24, 'Quercus'), (25, 'Quercus'), (26, 'Quercus'), (27, 'Quercus'), (28, 'Quercus'), (29, 'Quercus'), (30, 'Quercus'), (31, 'Quercus'), (32, 'Quercus'), (33, 'Quercus'), (34, 'Quercus'), (35, 'Quercus'), (36, 'Quercus'), (37, 'Quercus'), (38, 'Quercus'), (39, 'Quercus'), (40, 'Quercus'), (41, 'Quercus'), (42, 'Quercus'), (43, 'Quercus'), (44, 'Quercus'), (45, 'Quercus'), (46, 'Quercus'), (47, 'Quercus'), (48, 'Quercus'), (49, 'Quercus'), (50, 'Quercus'), (51, 'Quercus'), (52, 'Quercus'), (53, 'Quercus'), (54, 'Quercus'), (55, 'Quercus'), (56, 'Quercus')])


In [24]:
def get_acc():
  output_file = open("topk_ids.csv")
  output_lines = output_file.readlines()

  corr = 0
  total = 0
  for line in output_lines:
    
    line = line.strip().split(",")
    cat = class_dict[line[0]]
    pred = int(line[1])

    if cat == pred:
      corr += 1

    total += 1

  return corr / total

def sum_wrong(results_list):
  total = 0
  for v in results_list:
    if v == 0:
      total += 1

  return total


def acc_stats():
    species_wrong = defaultdict(list)
    genera_wrong = defaultdict(list)
    output_file = open("topk_ids.csv")
    output_lines = output_file.readlines()

    corr = 0
    total = 0
    for line in output_lines:
      line = line.strip().split(",")
      cat = class_dict[line[0]]
      gen = genera_dict[cat]
      pred = int(line[1])

      if cat == pred:
        corr += 1
        species_wrong[cat].append(1)
        genera_wrong[gen].append(1)
      else:
        species_wrong[cat].append(0)
        genera_wrong[gen].append(0)

    species_wrong = {k: sum_wrong(v) / len(v) for k, v in sorted(species_wrong.items(), key=lambda item: sum_wrong(item[1]) / len(item[1]), reverse=True)}
    genera_wrong = {k: sum_wrong(v) / len(v) for k, v in sorted(genera_wrong.items(), key=lambda item: sum_wrong(item[1]) / len(item[1]), reverse=True)}

    print([(species_dict.get(v[0]), v[1]) for v in list(species_wrong.items())[:20]])
    print(list(genera_wrong.items())[:20])



def balanced_acc():
    species_wrong = defaultdict(list)
    genera_wrong = defaultdict(list)
    output_file = open("topk_ids.csv")
    output_lines = output_file.readlines()

    corr = 0
    total = 0
    for line in output_lines:
      line = line.strip().split(",")
      cat = class_dict[line[0]]
      gen = genera_dict[cat]
      pred = int(line[1])

      if cat == pred:
        corr += 1
        species_wrong[cat].append(1)
        genera_wrong[gen].append(1)
      else:
        species_wrong[cat].append(0)
        genera_wrong[gen].append(0)

    species_wrong = {k: v[:15] for k, v in list(species_wrong.items())}
    out_lists = []
    for v in species_wrong.values():
      out_lists += v

    print(sum(out_lists) / len(out_lists))

In [20]:
# CNL Q600G4F15 Epoch 31/80
print(get_acc())

0.8415065869653532


In [23]:
acc_stats()

[('Quercus buckleyi Nixon & Dorr', 1.0), ('Quercus robur subsp. brutia (Ten.) O.Schwarz', 1.0), ('Quercus shumardii var. shumardii', 1.0), ('Quercus macrocarpa var. depressa (Nutt.) Engelm.', 0.8333333333333334), ('Quercus acerifolia (E.J.Palmer) Stoynoff & Hess', 0.6), ('Quercus boyntonii Beadle', 0.6), ('Quercus robur subsp. imeretina (Steven ex Woronow) Menitsky', 0.6), ('Quercus fusiformis Small', 0.5581395348837209), ('Quercus shumardii var. schneckii (Britton) Sarg.', 0.5517241379310345), ('Quercus texana Buckley', 0.5338345864661654), ('Quercus robur subsp. pedunculiflora (K.Koch) Menitsky', 0.4878048780487805), ('Quercus hemisphaerica W.Bartram ex Willd.', 0.4583333333333333), ('Quercus similis Ashe', 0.45454545454545453), ('Quercus austrina Small', 0.3333333333333333), ('Quercus shumardii Buckley', 0.31952662721893493), ('Quercus coccinea MÃ¼nchh.', 0.3151183970856102), ('Quercus minima (Sarg.) Small', 0.3020833333333333), ('Quercus geminata Small', 0.2838709677419355), ('Quer

In [25]:
balanced_acc()

0.7443037974683544
